In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
import numpy as np
import cv2
import os

In [3]:
DIRECTORY = os.path.join(os.getcwd(), 'dogscats', 'train')
CATEGORIES = ['cats', 'dogs', 'cavallo', 'elefante', 'farfalla', 'gallina', 'mucca', 'panda', 'pecora', 'ragno', 'scoiattolo']

In [4]:
data = []
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path)
        arr = cv2.resize(arr, (224, 224))  # Đặt kích thước ảnh phù hợp với VGG16
        data.append([arr, label])

In [5]:
# Shuffle dữ liệu
import random
random.shuffle(data)

In [6]:
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
# Chuẩn hóa dữ liệu
X = X / 255.0
X = X.reshape(-1, 224, 224, 3)  # Đảm bảo có 3 kênh (RGB)

MemoryError: Unable to allocate 49.0 GiB for an array with shape (43648, 224, 224, 3) and data type float64

In [9]:
# Tăng cường dữ liệu
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X)

MemoryError: Unable to allocate 24.5 GiB for an array with shape (43648, 224, 224, 3) and data type float32

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(CATEGORIES), activation='softmax'))

# Freeze các lớp của VGG16 để chỉ huấn luyện các lớp thêm
base_model.trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(datagen.flow(X, y, batch_size=32), epochs=20, validation_split=0.3)
model.save("model_trained_vgg16.h5")